In [1]:
import pandas as pd

# نام فایل خودت را جایگزین کن
df = pd.read_csv("Data\Posts_ikea_16_Oct_2025_14_Nov_2025_1b0e.csv")

print(df.head())
print(df.columns)


   Name                         Profile  \
0  ikea  https://www.instagram.com/ikea   
1  ikea  https://www.instagram.com/ikea   
2  ikea  https://www.instagram.com/ikea   
3  ikea  https://www.instagram.com/ikea   
4  ikea  https://www.instagram.com/ikea   

                                     Permalink  \
0     https://www.instagram.com/p/DQWZuZGkz3P/   
1     https://www.instagram.com/p/DQuCeK8ieO3/   
2     https://www.instagram.com/p/DQq9yeUjQxz/   
3  https://www.instagram.com/reel/DQL8rHXgE1B/   
4  https://www.instagram.com/reel/DP3dZNHABfm/   

                                             Caption            Type  \
0  A cosy corner for when you need a calm moment ...           IMAGE   
1  Smart products now simpler than ever ✨⁠⁠We’re ...  CAROUSEL_ALBUM   
2  Making waste sorting a little easier ✨ Say hej...  CAROUSEL_ALBUM   
3  For everyone at the table kids grown-ups and e...            reel   
4  It’s not about perfect plates or fancy recipes...            reel   

       

In [2]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


In [3]:
from transformers import pipeline

sentiment_model = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment"  # خروجی 1 تا 5 ستاره
)


C:\Users\Nastaran\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
Device set to use cpu


In [4]:
df['Caption'] = df['Caption'].fillna("").astype(str)

results = sentiment_model(df['Caption'].tolist())

df['bert_sentiment_label'] = [r['label'] for r in results]
df['bert_sentiment_score'] = [r['score'] for r in results]

df[['Caption', 'bert_sentiment_label', 'bert_sentiment_score']].head()


,Caption,bert_sentiment_label,bert_sentiment_score
0,A cosy corner for when you need a calm moment ...,4 stars,0.513128
1,Smart products now simpler than ever ✨⁠⁠We’re ...,5 stars,0.849247
2,Making waste sorting a little easier ✨ Say hej...,5 stars,0.544678
3,For everyone at the table kids grown-ups and e...,5 stars,0.509101
4,It’s not about perfect plates or fancy recipes...,3 stars,0.490920


In [5]:
def stars_to_polarity(label):
    n = int(label.split()[0])  # "4 stars" → 4
    if n <= 2:
        return "negative"
    elif n == 3:
        return "neutral"
    else:
        return "positive"

df['bert_sentiment_polarity'] = df['bert_sentiment_label'].apply(stars_to_polarity)

df[['Caption', 'bert_sentiment_label', 'bert_sentiment_polarity']].head()


,Caption,bert_sentiment_label,bert_sentiment_polarity
0,A cosy corner for when you need a calm moment ...,4 stars,positive
1,Smart products now simpler than ever ✨⁠⁠We’re ...,5 stars,positive
2,Making waste sorting a little easier ✨ Say hej...,5 stars,positive
3,For everyone at the table kids grown-ups and e...,5 stars,positive
4,It’s not about perfect plates or fancy recipes...,3 stars,neutral


In [6]:
from keybert import KeyBERT
kw_model = KeyBERT(model='all-MiniLM-L6-v2')


In [7]:
import re

def caption_to_hashtags(text, top_n=5):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    
    keywords = kw_model.extract_keywords(
        text,
        keyphrase_ngram_range=(1, 2),  # تک‌کلمه‌ای و دوکلمه‌ای
        stop_words='english',
        top_n=top_n
    )
    
    tags = []
    for kw, score in keywords:
        # تمیز کردن کلمه
        kw_clean = re.sub(r'[^a-zA-Z0-9\s]', '', kw)
        if not kw_clean:
            continue
        tag = "#" + kw_clean.replace(" ", "").lower()
        tags.append(tag)
    
    # حذف تکراری‌ها با حفظ ترتیب
    tags = list(dict.fromkeys(tags))
    return " ".join(tags)

df['suggested_hashtags'] = df['Caption'].apply(caption_to_hashtags)

df[['Caption', 'suggested_hashtags']].head()


,Caption,suggested_hashtags
0,A cosy corner for when you need a calm moment ...,#cosycorner #calmmoment #cosy #hemnes #needcalm
1,Smart products now simpler than ever ✨⁠⁠We’re ...,#smarthome #smartproducts #smarttechnology #re...
2,Making waste sorting a little easier ✨ Say hej...,#wastecollector #wastesorting #organicwaste #m...
3,For everyone at the table kids grown-ups and e...,#plantballs #differentballs #balls #ballstried...
4,It’s not about perfect plates or fancy recipes...,#cookinghome #recipesmoments #perfectplates #f...


In [8]:
df.groupby('bert_sentiment_polarity')['Likes'].mean()


bert_sentiment_polarity
neutral     741.000
positive    941.125
Name: Likes, dtype: float64

In [9]:
df.groupby('Type')['Engagement'].mean()


Type
CAROUSEL_ALBUM    1487.0
IMAGE              920.0
reel               727.4
Name: Engagement, dtype: float64

In [10]:
df.groupby('Content Pillars')['Eng. Rate by Followers'].mean()


Series([], Name: Eng. Rate by Followers, dtype: float64)

In [11]:
from transformers import pipeline

# مدل مخصوص سوشال‌مدیا با 3 برچسب: negative / neutral / positive
sentiment_model = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest"
)

# مطمئن می‌شویم کپشن‌ها رشته‌ی خالی نباشند
df['Caption'] = df['Caption'].fillna("").astype(str)

results = sentiment_model(df['Caption'].tolist())

# اضافه کردن به دیتافریم
df['sentiment_label'] = [r['label'] for r in results]   # NEGATIVE / NEUTRAL / POSITIVE
df['sentiment_score'] = [r['score'] for r in results]   # میزان اطمینان مدل

df[['Caption', 'sentiment_label', 'sentiment_score']].head()


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

C:\Users\Nastaran\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nastaran\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment-latest. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(me

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Device set to use cpu


,Caption,sentiment_label,sentiment_score
0,A cosy corner for when you need a calm moment ...,positive,0.791091
1,Smart products now simpler than ever ✨⁠⁠We’re ...,positive,0.937169
2,Making waste sorting a little easier ✨ Say hej...,positive,0.766185
3,For everyone at the table kids grown-ups and e...,positive,0.714313
4,It’s not about perfect plates or fancy recipes...,positive,0.950000


In [13]:
df = df.drop(columns=['bert_sentiment_label', 'bert_sentiment_score', 'bert_sentiment_polarity'], errors='ignore')


In [14]:
df.groupby('sentiment_label')['Likes'].mean()


sentiment_label
positive    918.888889
Name: Likes, dtype: float64

In [15]:
df['sentiment_label'].value_counts()


sentiment_label
positive    9
Name: count, dtype: int64